# 🔢 Phase-2.3: Vector Generation
## Quantum-RAG Knowledge Fusion for Adaptive IoT Intrusion Detection

---

### 📋 Phase-2.3 Objective

**Convert encoded features into fixed-length vectors suitable for ChromaDB ingestion.**

Phase-2.3 Goals:
1. ✅ Load encoded features from Phase-2.2
2. ✅ Load metadata labels (attack types)
3. ✅ Create fixed-length feature vectors (99-dimensional)
4. ✅ Validate vector integrity and dimensionality
5. ✅ Generate vector IDs and metadata
6. ✅ Save vectors in ChromaDB-compatible format

### 🎯 Vector Requirements

| Requirement | Specification |
|-------------|---------------|
| **Dimensions** | 99 (from Phase-2.2 encoding) |
| **Data Type** | float32 (memory efficient) |
| **Metadata** | label (attack type), type (Normal/Attack) |
| **Format** | NumPy arrays + Parquet for metadata |
| **Validation** | No NaN, no Inf, all finite values |

### 📊 Key Principles

- **Chunked Processing**: Handle 22.3M vectors efficiently
- **Memory Management**: Process in batches to avoid RAM exhaustion
- **Type Safety**: Ensure float32 throughout pipeline
- **Metadata Integrity**: Preserve attack labels for retrieval

---

## 📦 Import Required Libraries

In [16]:
# Core data manipulation
import pandas as pd
import numpy as np
import json
import gc

# File handling
import os
from pathlib import Path

# Progress tracking
from tqdm import tqdm

# Display utilities
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Define paths
PHASE_2_DIR = "../artifacts/phase_2"

print("✅ Libraries imported successfully!")
print(f"📁 Phase-2 artifacts: {PHASE_2_DIR}")

✅ Libraries imported successfully!
📁 Phase-2 artifacts: ../artifacts/phase_2


---

## 🔒 SECTION 1 — Load Encoded Features & Metadata

### Objectives:
1. Load encoded features from Phase-2.2
2. Load metadata labels (attack types)
3. Validate data alignment
4. Check dimensionality

In [17]:
# Load Phase-2.2 summary
with open(f"{PHASE_2_DIR}/phase2_2_summary.json", 'r', encoding='utf-8') as f:
    phase2_2_summary = json.load(f)

print("📊 Phase-2.2 Summary:")
print(f"  • Input Rows: {phase2_2_summary['input_shape'][0]:,}")
print(f"  • Encoded Features: {phase2_2_summary['output_shape'][1]}")
print(f"  • Feature Expansion: {phase2_2_summary['feature_expansion']['expansion_factor']}x")
print(f"  • All Numeric: {phase2_2_summary['validation']['all_numeric']}")

📊 Phase-2.2 Summary:
  • Input Rows: 22,339,021
  • Encoded Features: 99
  • Feature Expansion: 3.0x
  • All Numeric: True


In [18]:
# Load encoded features (Parquet is much faster than CSV)
print("📥 Loading encoded features from Phase-2.2...\n")

encoded_features = pd.read_parquet(f"{PHASE_2_DIR}/encoded_features.parquet")

print(f"✅ Loaded:")
print(f"  • Rows: {len(encoded_features):,}")
print(f"  • Columns: {len(encoded_features.columns)}")
print(f"  • Memory: {encoded_features.memory_usage(deep=True).sum() / (1024**3):.2f} GB")
print(f"  • Data Types: {encoded_features.dtypes.value_counts().to_dict()}")

📥 Loading encoded features from Phase-2.2...

✅ Loaded:
  • Rows: 22,339,021
  • Columns: 99
  • Memory: 2.85 GB
  • Data Types: {dtype('uint8'): 83, dtype('float32'): 12, dtype('int16'): 2, dtype('int8'): 2}


In [19]:
# Load metadata labels
print("\n📥 Loading metadata labels...\n")

metadata_df = pd.read_csv(f"{PHASE_2_DIR}/metadata_labels.csv.gz", compression='gzip')

print(f"✅ Loaded:")
print(f"  • Rows: {len(metadata_df):,}")
print(f"  • Columns: {list(metadata_df.columns)}")

# Check attack type distribution (type column has actual attack names)
print(f"\n📊 Attack Type Distribution:")
type_dist = metadata_df['type'].value_counts()
for attack_type, count in type_dist.items():
    print(f"  • {attack_type}: {count:,} ({count / len(metadata_df) * 100:.1f}%)")

print(f"\n📊 Binary Label Distribution:")
print(f"  • Normal (0): {(metadata_df['label'] == 0).sum():,}")
print(f"  • Attack (1): {(metadata_df['label'] == 1).sum():,}")


📥 Loading metadata labels...

✅ Loaded:
  • Rows: 22,339,021
  • Columns: ['label', 'type']

📊 Attack Type Distribution:
  • scanning: 7,140,161 (32.0%)
  • ddos: 6,165,008 (27.6%)
  • dos: 3,375,328 (15.1%)
  • xss: 2,108,944 (9.4%)
  • password: 1,718,568 (7.7%)
  • normal: 796,380 (3.6%)
  • backdoor: 508,116 (2.3%)
  • injection: 452,659 (2.0%)
  • ransomware: 72,805 (0.3%)
  • mitm: 1,052 (0.0%)

📊 Binary Label Distribution:
  • Normal (0): 796,380
  • Attack (1): 21,542,641


In [20]:
# Validate data alignment
print("🔍 Validating data alignment...\n")

if len(encoded_features) != len(metadata_df):
    raise ValueError(f"Row count mismatch! Encoded: {len(encoded_features):,}, Metadata: {len(metadata_df):,}")

print("✅ Data alignment verified!")
print(f"  • Both datasets: {len(encoded_features):,} rows")
print(f"  • Vector dimensions: {encoded_features.shape[1]}")

🔍 Validating data alignment...

✅ Data alignment verified!
  • Both datasets: 22,339,021 rows
  • Vector dimensions: 99


---

## 🔢 SECTION 2 — Vector Generation

### Objectives:
1. Convert DataFrame rows to NumPy vectors
2. Validate vector integrity (no NaN, no Inf)
3. Ensure float32 data type
4. Generate unique vector IDs

In [21]:
# Convert to NumPy array (vectors)
print("🔢 Converting encoded features to vectors...\n")

# Ensure all columns are float32
for col in encoded_features.columns:
    if encoded_features[col].dtype != np.float32:
        encoded_features[col] = encoded_features[col].astype(np.float32)

# Convert to NumPy array
vectors = encoded_features.values

print(f"✅ Vectors created:")
print(f"  • Shape: {vectors.shape}")
print(f"  • Data Type: {vectors.dtype}")
print(f"  • Memory: {vectors.nbytes / (1024**3):.2f} GB")

🔢 Converting encoded features to vectors...

✅ Vectors created:
  • Shape: (22339021, 99)
  • Data Type: float32
  • Memory: 8.24 GB


In [22]:
# Validate vector integrity
print("\n🔍 Validating vector integrity...\n")

# Check for NaN
nan_count = np.isnan(vectors).sum()
if nan_count > 0:
    print(f"⚠️  NaN values detected: {nan_count:,}")
else:
    print("✅ No NaN values detected")

# Check for Inf
inf_count = np.isinf(vectors).sum()
if inf_count > 0:
    print(f"⚠️  Inf values detected: {inf_count:,}")
else:
    print("✅ No Inf values detected")

# Check if all finite
all_finite = np.isfinite(vectors).all()
if all_finite:
    print("✅ All values are finite")
else:
    print("⚠️  Some values are not finite")

# Vector statistics
print(f"\n📊 Vector Statistics:")
print(f"  • Min value: {vectors.min():.6f}")
print(f"  • Max value: {vectors.max():.6f}")
print(f"  • Mean: {vectors.mean():.6f}")
print(f"  • Std: {vectors.std():.6f}")


🔍 Validating vector integrity...

✅ No NaN values detected
✅ No Inf values detected
✅ All values are finite

📊 Vector Statistics:
  • Min value: -32768.000000
  • Max value: 541129.562500
  • Mean: 0.723042
  • Std: 58.053623


In [23]:
# Generate unique vector IDs
print("\n🆔 Generating unique vector IDs...\n")

# Use sequential IDs for 22M+ vectors
vector_ids = [f"vec_{i:08d}" for i in range(len(vectors))]

print(f"✅ Generated {len(vector_ids):,} vector IDs")
print(f"  • Format: vec_00000000 to vec_{len(vector_ids)-1:08d}")
print(f"  • Example IDs: {vector_ids[:3]}")


🆔 Generating unique vector IDs...

✅ Generated 22,339,021 vector IDs
  • Format: vec_00000000 to vec_22339020
  • Example IDs: ['vec_00000000', 'vec_00000001', 'vec_00000002']


---

## 📋 SECTION 3 — Metadata Preparation

### Objectives:
1. Create metadata dictionary for each vector
2. Include attack label and type
3. Prepare ChromaDB-compatible metadata format

In [24]:
# Prepare metadata for ChromaDB
print("📋 Preparing metadata for ChromaDB...\n")

# Add vector IDs to metadata
metadata_df['vector_id'] = vector_ids

# Verify the 'type' column already contains attack type strings
# (from Phase-2.1: 'normal', 'scanning', 'ddos', 'dos', 'xss', 'password', 
#  'backdoor', 'injection', 'ransomware', 'mitm')
print(f"✅ Metadata prepared:")
print(f"  • Rows: {len(metadata_df):,}")
print(f"  • Columns: {list(metadata_df.columns)}")

print(f"\n📊 Attack Type Distribution:")
print(metadata_df['type'].value_counts())

print(f"\n📊 Binary Label Distribution:")
print(f"  • Normal (0): {(metadata_df['label'] == 0).sum():,}")
print(f"  • Attack (1): {(metadata_df['label'] == 1).sum():,}")

📋 Preparing metadata for ChromaDB...

✅ Metadata prepared:
  • Rows: 22,339,021
  • Columns: ['label', 'type', 'vector_id']

📊 Attack Type Distribution:
type
scanning      7140161
ddos          6165008
dos           3375328
xss           2108944
password      1718568
normal         796380
backdoor       508116
injection      452659
ransomware      72805
mitm             1052
Name: count, dtype: int64

📊 Binary Label Distribution:
  • Normal (0): 796,380
  • Attack (1): 21,542,641


In [25]:
# Preview sample vectors with metadata
print("\n🔍 Sample Vectors with Metadata:")
print("="*80)

sample_indices = [0, 1, len(vectors)//2, len(vectors)-2, len(vectors)-1]
for idx in sample_indices:
    print(f"\nVector ID: {vector_ids[idx]}")
    print(f"  Binary Label: {metadata_df.iloc[idx]['label']} ({'Normal' if metadata_df.iloc[idx]['label'] == 0 else 'Attack'})")
    print(f"  Attack Type: {metadata_df.iloc[idx]['type']}")
    print(f"  Vector shape: {vectors[idx].shape}")
    print(f"  First 5 values: {vectors[idx][:5]}")


🔍 Sample Vectors with Metadata:

Vector ID: vec_00000000
  Binary Label: 0 (Normal)
  Attack Type: normal
  Vector shape: (99,)
  First 5 values: [-1.8826956e+00  3.3064134e+00  4.6609456e+05  0.0000000e+00
 -1.0000000e+00]

Vector ID: vec_00000001
  Binary Label: 0 (Normal)
  Attack Type: normal
  Vector shape: (99,)
  First 5 values: [ 0.46473706  0.64512044 -0.00116886  0.         -1.        ]

Vector ID: vec_11169510
  Binary Label: 1 (Attack)
  Attack Type: scanning
  Vector shape: (99,)
  First 5 values: [ 2.2780557e-01  1.6872513e+00 -1.1688597e-03  0.0000000e+00
 -1.0000000e+00]

Vector ID: vec_22339019
  Binary Label: 1 (Attack)
  Attack Type: dos
  Vector shape: (99,)
  First 5 values: [-1.8223245e+00 -2.4848045e-01 -6.5965345e-04  0.0000000e+00
 -1.0000000e+00]

Vector ID: vec_22339020
  Binary Label: 1 (Attack)
  Attack Type: dos
  Vector shape: (99,)
  First 5 values: [-1.8223245e+00 -2.4848045e-01 -3.8769108e-04  0.0000000e+00
 -1.0000000e+00]


---

## 💾 SECTION 4 — Save Vectors & Metadata

### Objectives:
1. Save vectors as NumPy binary (.npy)
2. Save metadata as Parquet
3. Generate summary statistics
4. Validate saved files

In [26]:
# Save vectors as NumPy binary
print("💾 Saving vectors...\n")

vectors_path = f"{PHASE_2_DIR}/feature_vectors.npy"
np.save(vectors_path, vectors)

print(f"✅ Saved: {vectors_path}")
print(f"  • Vectors: {vectors.shape[0]:,}")
print(f"  • Dimensions: {vectors.shape[1]}")
print(f"  • Size: {os.path.getsize(vectors_path) / (1024**3):.2f} GB")

💾 Saving vectors...

✅ Saved: ../artifacts/phase_2/feature_vectors.npy
  • Vectors: 22,339,021
  • Dimensions: 99
  • Size: 8.24 GB


In [27]:
# Save metadata as Parquet
print("\n💾 Saving metadata...\n")

metadata_path = f"{PHASE_2_DIR}/vector_metadata.parquet"
metadata_df.to_parquet(
    metadata_path,
    index=False,
    compression='snappy',
    engine='pyarrow'
)

print(f"✅ Saved: {metadata_path}")
print(f"  • Rows: {len(metadata_df):,}")
print(f"  • Columns: {list(metadata_df.columns)}")
print(f"  • Size: {os.path.getsize(metadata_path) / (1024**2):.1f} MB")


💾 Saving metadata...

✅ Saved: ../artifacts/phase_2/vector_metadata.parquet
  • Rows: 22,339,021
  • Columns: ['label', 'type', 'vector_id']
  • Size: 108.9 MB


In [28]:
# Generate Phase-2.3 summary
summary = {
    "phase": "Phase-2.3 (Vector Generation)",
    "timestamp": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
    "input_files": {
        "encoded_features": "encoded_features.parquet",
        "metadata_labels": "metadata_labels.csv.gz"
    },
    "output_files": {
        "vectors": "feature_vectors.npy",
        "metadata": "vector_metadata.parquet"
    },
    "vectors": {
        "total_count": int(vectors.shape[0]),
        "dimensions": int(vectors.shape[1]),
        "data_type": str(vectors.dtype),
        "memory_gb": round(vectors.nbytes / (1024**3), 2)
    },
    "statistics": {
        "min_value": float(vectors.min()),
        "max_value": float(vectors.max()),
        "mean_value": float(vectors.mean()),
        "std_value": float(vectors.std())
    },
    "validation": {
        "nan_count": int(nan_count),
        "inf_count": int(inf_count),
        "all_finite": bool(all_finite)
    },
    "metadata": {
        "total_rows": int(len(metadata_df)),
        "normal_count": int((metadata_df['label'] == 0).sum()),
        "attack_count": int((metadata_df['label'] == 1).sum()),
        "unique_attack_types": int(metadata_df['type'].nunique()),
        "attack_type_distribution": metadata_df['type'].value_counts().to_dict()
    }
}

summary_path = f"{PHASE_2_DIR}/phase2_3_summary.json"
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2)

print("\n📊 Phase-2.3 Summary:")
print(json.dumps(summary, indent=2))
print(f"\n💾 Saved: {summary_path}")


📊 Phase-2.3 Summary:
{
  "phase": "Phase-2.3 (Vector Generation)",
  "timestamp": "2026-02-04 22:03:49",
  "input_files": {
    "encoded_features": "encoded_features.parquet",
    "metadata_labels": "metadata_labels.csv.gz"
  },
  "output_files": {
    "vectors": "feature_vectors.npy",
    "metadata": "vector_metadata.parquet"
  },
  "vectors": {
    "total_count": 22339021,
    "dimensions": 99,
    "data_type": "float32",
    "memory_gb": 8.24
  },
  "statistics": {
    "min_value": -32768.0,
    "max_value": 541129.5625,
    "mean_value": 0.7230424880981445,
    "std_value": 58.05362319946289
  },
  "validation": {
    "nan_count": 0,
    "inf_count": 0,
    "all_finite": true
  },
  "metadata": {
    "total_rows": 22339021,
    "normal_count": 796380,
    "attack_count": 21542641,
    "unique_attack_types": 10,
    "attack_type_distribution": {
      "scanning": 7140161,
      "ddos": 6165008,
      "dos": 3375328,
      "xss": 2108944,
      "password": 1718568,
      "normal": 7

In [29]:
# Validate saved files
print("\n🔍 Validating saved files...\n")

# Load and validate vectors
loaded_vectors = np.load(vectors_path)
print(f"✅ Vectors loaded successfully: {loaded_vectors.shape}")

# Load and validate metadata
loaded_metadata = pd.read_parquet(metadata_path)
print(f"✅ Metadata loaded successfully: {loaded_metadata.shape}")

# Verify integrity
vectors_match = np.array_equal(vectors, loaded_vectors)
metadata_match = metadata_df.equals(loaded_metadata)

if vectors_match and metadata_match:
    print("\n✅ All saved files validated successfully!")
else:
    print("\n⚠️  Validation failed!")
    print(f"  • Vectors match: {vectors_match}")
    print(f"  • Metadata match: {metadata_match}")


🔍 Validating saved files...

✅ Vectors loaded successfully: (22339021, 99)
✅ Metadata loaded successfully: (22339021, 3)

✅ All saved files validated successfully!


---

## 🎉 Phase-2.3 Complete!

### ✅ Deliverables
1. ✅ Generated 22.3M fixed-length vectors (99 dimensions)
2. ✅ Validated vector integrity (no NaN, no Inf)
3. ✅ Prepared metadata with attack labels
4. ✅ Saved vectors as NumPy binary (.npy)
5. ✅ Saved metadata as Parquet
6. ✅ Generated summary statistics

### 📂 Output Files
- `artifacts/phase_2/feature_vectors.npy` (~8.3 GB, 22.3M × 99 float32)
- `artifacts/phase_2/vector_metadata.parquet` (~50 MB, vector IDs + labels)
- `artifacts/phase_2/phase2_3_summary.json`

### 🚀 Next: Phase-2.4 (ChromaDB Ingestion)
Ready to:
- Initialize ChromaDB collection
- Batch insert 22.3M vectors with metadata
- Configure similarity search parameters
- Validate ingestion completeness

---

**Status**: ✅ **READY FOR PHASE-2.4**